In [1]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import os

In [2]:
class CNN(Module):
	def __init__(self, numChannels, classes):
		# call the parent constructor
		super(CNN, self).__init__()
		# initialize first set of CONV => RELU =>CONV => RELU => POOL layers
		self.conv11 = Conv2d(in_channels=numChannels, out_channels=32,
			kernel_size=(3, 3))
		self.relu11 = ReLU()
		self.conv12 = Conv2d(in_channels=32, out_channels=32,
			kernel_size=(3, 3))
		self.relu12 = ReLU()
		self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

		# initialize second set of CONV => RELU => CONV => RELU => POOL layers
		self.conv21 = Conv2d(in_channels=32, out_channels=64,
			kernel_size=(5, 5))
		self.relu21 = ReLU()
		self.conv22 = Conv2d(in_channels=64, out_channels=64,
			kernel_size=(3, 3))
		self.relu22 = ReLU()
		self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

		# initialize third set of CONV => RELU => CONV => RELU => POOL layers
		self.conv31 = Conv2d(in_channels=64, out_channels=128,
			kernel_size=(5, 5))
		self.relu31 = ReLU()
		self.conv32 = Conv2d(in_channels=128, out_channels=128,
			kernel_size=(3, 3))
		self.relu32 = ReLU()
		self.maxpool3 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

		# initialize first set of FC => RELU layers
		self.fc1 = Linear(in_features=3*3*128, out_features=5*128)
		self.relu3 = ReLU()

		# initialize second set of FC => RELU layers
		self.fc1 = Linear(in_features=5*128, out_features=5*128)
		self.relu3 = ReLU()

		# initialize our softmax classifier
		self.fc2 = Linear(in_features=5*128, out_features=classes)
		self.logSoftmax = LogSoftmax(dim=1) #dim=1 je ze to robi po riadkoch

	def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
		x = self.conv11(x)
		x = self.relu11(x)
		x = self.conv12(x)
		x = self.relu12(x)
		x = self.maxpool1(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv21(x)
		x = self.relu21(x)
		x = self.conv22(x)
		x = self.relu22(x)
		x = self.maxpool2(x)
		# pass the output from the previous layer through the third
		# set of CONV => RELU => POOL layers
		x = self.conv31(x)
		x = self.relu31(x)
		x = self.conv32(x)
		x = self.relu32(x)
		x = self.maxpool3(x)
		# flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
		x = flatten(x, 1)
		x = self.fc1(x)
		x = self.relu3(x)
		# pass the output to our softmax classifier to get our output
		# predictions
		x = self.fc2(x)
		output = self.logSoftmax(x)
		# return the output predictions
		return output


In [3]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
#from pyimagesearch.lenet import LeNet
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
#from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time


In [4]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
#https://medium.com/thecyphy/train-cnn-model-with-pytorch-21dafb918f48
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
path_train = 'D:\\skola\\ING\\LS2023\\NSIETE\\zadanie2_data\\train\\'
path_val = 'D:\\skola\\ING\\LS2023\\NSIETE\\zadanie2_data\\val\\'
path_test = 'D:\\skola\\ING\\LS2023\\NSIETE\\zadanie2_data\\test\\'

dataset_train = ImageFolder(path_train,transforms.Compose([
    transforms.ToTensor()
]))
dataset_val = ImageFolder(path_val,transforms.Compose([
   transforms.ToTensor()
]))
dataset_test = ImageFolder(path_test,transforms.Compose([
    transforms.ToTensor()
]))

In [6]:
# initialize the LeNet model
print("[INFO] initializing the LeNet model...")
model = CNN(
	numChannels=3,
	classes=4) #len(train_imgs.dataset.classes)).to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

[INFO] initializing the LeNet model...
[INFO] training the network...


In [7]:
# initialize the train, validation, and test data loaders
trainDataLoader = DataLoader(dataset_train, shuffle=True,
	batch_size=BATCH_SIZE)
valDataLoader = DataLoader(dataset_val, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(dataset_test, batch_size=BATCH_SIZE)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [8]:
trainDataLoader

In [12]:
# loop over our epochs
for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in trainDataLoader:
		print("x: ",len(x)," y:",len(y))
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()


x:  64  y: 64


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x1152 and 640x640)

In [14]:
# switch off autograd for evaluationx115
with torch.no_grad():
	# set the model in evaluation mode
	model.eval()
	# loop over the validation set
	for (x, y) in valDataLoader:
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# make the predictions and calculate the validation loss
		pred = model(x)
		totalValLoss += lossFn(pred, y)
		# calculate the number of correct predictions
		valCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x1152 and 640x640)